## Reinforcement Learning for SMS Messaging to Improve Medication Adherence - Roybal

In [27]:
import sys
import time
import dateutil
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankRequest
from msrest.authentication import CognitiveServicesCredentials
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime
from collections import Counter
import string
import pickle
import json
import pytz
import os

In [11]:
from fresh_restart.patient_data import import_pt_data,export_pt_data
from fresh_restart.pillsy_parser import import_Pillsy, find_rewards
from fresh_restart.driverReward import get_reward_update,send_rewards
from fresh_restart.redcap_parser import import_redcap, update_pt_data_with_redcap
from fresh_restart.driverRank import run_ranking, write_sms_history
from fresh_restart.control_disconnection import check_control_disconnectedness

### Start Program Timer

In [24]:
run_time = datetime.now()
testing_flag = input("Testing with another date? y/n: ")
if testing_flag.lower() == "y":
    print("Set testing run time: ")
    run_time_yyyy_mm_dd_input = input("Enter the testing date: YYYY-MM-DD ")
    timestamp = "10:30 AM " + run_time_yyyy_mm_dd_input
    run_time = dateutil.parser.parse(timestamp)
run_time = pytz.timezone("America/New_York").localize(run_time)


Testing with another date? y/n: y
Set testing run time: 
Enter the testing date: YYYY-MM-DD 2020-11-23


In [25]:
run_time

datetime.datetime(2020, 11, 23, 10, 30, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)

### Set Up MS Azure Personalizer Client

Defining and Instantiating a Personalizer Client:

Personalizer Keys:
* In the Microsoft Azure Dashboard, navigate to our bwh-pharmacoepi-roybal-dev-use2-cog Cognitive Services page.
* Within the Keys and Endpoint section, copy either Key 1 or Key 2 to enter as the Personalizer Key.

Personalizer Endpoint:
* https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/

In [15]:
## TODO make this a relative path that works for both our machines
personalizer_path = input("ENTER FILEPATH TO PERSONALIZER KEY: \n")
with open(personalizer_path, 'r') as f:
     personalizer_key = f.read()
personalizer_endpoint = "https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/"
client = PersonalizerClient(personalizer_endpoint, CognitiveServicesCredentials(personalizer_key))

ENTER FILEPATH TO PERSONALIZER KEY: 
C:\Users\lg436\.keys\azure-personalizer-key.txt


## Reward Step

If we've already initiated the trial, we will have:
* Pre-existing patient dataset in need of reward updates
* Pillsy data from yesterday to determine reward
If this is study initiation, this step will just load an empty patient dictionary and null pillsy dataset.

In [ ]:
pt_data = import_pt_data(run_time)
new_pillsy_data = import_Pillsy(run_time)

if pt_data and new_pillsy_data:
    
    # From Pillsy data, computes the Rewards to send to Personalizer for each patient's Rank calls from yesterday's run.
    find_rewards(new_pillsy_data, pt_data, run_time)
    
    # using updated patient data (new pillsy + patient data), format the rewards to Personalizer into a dataframe
    rewards_to_send = get_reward_update(pt_data, run_time)
    
    # actual call to personalizer
    send_rewards(rewards_to_send, client, run_time)
    
export_pt_data(pt_data, "reward", run_time)

## Import/Update Patients

In [33]:
redcap_data = import_redcap(run_time)
pt_data = update_pt_data_with_redcap(redcap_data, pt_data, run_time)

TypeError: 'NoneType' object is not subscriptable

In [34]:
redcap_data = import_redcap(run_time)
print(redcap_data)

   record_id    age         sex num_years_dm_rx    hba1c  race___1  race___2  \
0         13  45-54  Not listed             1-2  7.5-8.0         1         0   
1         14  55-64           F             3-4  8.1-8.9         0         1   
2         15  75-84           M              5+      10+         0         0   
3         16  75-84  Not listed               0  8.1-8.9         0         0   
4         17  18-34           F             3-4  7.5-8.0         0         0   
5         18  35-44           F              5+  9.0-9.9         0         0   
6         19  65-74           M             3-4  7.5-8.0         0         0   
7         20  65-74           M               0  8.1-8.9         0         0   

   race___3  race___4  race___5  ...  pillsy_meds___3  pillsy_meds___4  \
0         0         0         0  ...                1                0   
1         0         0         0  ...                0                0   
2         1         0         0  ...                1    

In [32]:
# # testing import_redcap
# run_time_redcaptest = run_time

# import_date = run_time_redcaptest.date()
# # Imports REDCap patients that are enrolling on an ongoing basis as a pandas data frame from a CSV
# redcap_filepath = str(import_date) + "_redcap" + '.csv'
# fp = os.path.join("..", "..", "REDCap", redcap_filepath)
# date_cols = ["start_date"]
# redcap = pd.read_csv(fp, sep=',', parse_dates=date_cols)
# redcap = redcap_vars_converter(redcap)

# redcap
# print(os.getcwd())
# print(os.path.abspath(fp))

In [ ]:
# run_time = 

import_date = run_time.date()
# Imports REDCap patients that are enrolling on an ongoing basis as a pandas data frame from a CSV
redcap_filepath = str(import_date) + "_redcap" + '.csv'
fp = os.path.join("..", "..", "REDCap", redcap_filepath)
date_cols = ["start_date"]

## Rank Step
Call Personalizer to rank action features to find the correct text message to send today.

In [ ]:
for index, patient in pt_data.iterrows():
    if patient["censor"] != 1 and patient["censor_date"] > run_time.date():
        run_ranking(patient, client, run_time)
        #need checks if pass by ref or val
export_pt_data(pt_data, "rank", run_time) # log

## Output SMS and Patient Data

In [ ]:
write_sms_history(pt_data, run_time)
export_pt_data(pt_data, "final", run_time) # input for tomorrow
check_control_disconnectedness(run_time) # check whether controls have connection problems

To Do List:
  * Executability by RA / User friendliness
    * [x] - Lily done by using Jupyter notebook - Making this more user friendly than a  Command Line
    * [ ] - JOE TODO / help brainstorm - make jupyter notebook pretty and write a how to document for new non CS person to be able to execute this from Jupyter notebook
    * [ ] - JOE TODO / help brainstorm - Making this more user friendly than a Jupyter Notebook - to do by doing a main.py executable bash script
    * [ ] - JOE TODO / help brainstorm - Hooks into Pillsy/RedCap for data retrieval - need check in with constance for pillsy and ellen for redcap
    * [ ] - JOE TODO / help brainstorm - (probably not feasible) Hooks into SMS Platform to automate text sending - need check in with constance
    * [ ] - JOE TODO / help brainstorm - how to handle the definite 1 human entered variable of personalizer key - file to direct to or? - maybe as marco/elad
    * [ ] - Lily/JOE TODO / help - try to break this code in any way possible
    * [ ] - Lily/JOE TODO / help - debug, unit testing
    * [ ] - Lily/JOE TODO / help - run it fully several times based in dropbox together
    * [ ] - Lily/JOE TODO / help - Make a log file that will report high level information from running this like a run summary
        * elements to include:
            * start and end time of the run
            * how many patients were read in from each import statement
            * how many reward calls were successfull made
            * how many rank calls were successfull made
            * any other meta data that will help us debug and ensure this is all working as planned


